In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os
import cmocean as cm
import waypoint_distance as wd
import pandas as pd
from pathlib import Path
from datetime import datetime
from matplotlib.dates import DateFormatter
import gsw
import matplotlib.dates as mdates
%matplotlib widget

new_cube = xr.open_dataset(os.path.expanduser('~/Desktop/Summer 2025 Python/calvert_cube.nc'))
new_cube = new_cube.sel(transect=new_cube.transect != '20250317_out')
new_cube = new_cube.assign_coords(along=new_cube['along'] - 25500)
topo = xr.open_dataset(os.path.expanduser('~/Desktop/Summer 2025 Python/british_columbia_3_msl_2013.nc'))
ds = xr.open_dataset('~/Desktop/Summer 2025 Python/Hakai_calvert.nc')

def estimate_critical_flow_speed_all_in_month(cube, target_month, along_target, upper_layer_range=(0, 100), lower_layer_range=(100, 130)):
    import numpy as np
    import gsw
    from scipy.interpolate import interp1d

    results = []
    transects = cube.transect.values
    matching = [t for t in transects if str(target_month) in str(t)]
    if not matching:
        print(f"No transects found for month {target_month}")
        return []

    for transect in matching:
        ds = cube.sel(transect=transect)
        along = ds['along'].values
        idx = np.argmin(np.abs(along - along_target))

        T = ds['temperature'][:, idx].values
        S = ds['salinity'][:, idx].values
        depth = ds['depth'].values

        p = gsw.p_from_z(-depth, 51.5)
        rho = gsw.density.rho(S, T, p)

        # Masks for layers
        upper_mask = (depth >= upper_layer_range[0]) & (depth < upper_layer_range[1])
        lower_mask = (depth >= lower_layer_range[0]) & (depth < lower_layer_range[1])

        if np.sum(upper_mask) < 1 or np.sum(lower_mask) < 1:
            continue

        rho1 = np.nanmean(rho[upper_mask])
        rho2 = np.nanmean(rho[lower_mask])

        g = 9.81
        g_prime = g * (rho2 - rho1) / rho2
        h = lower_layer_range[1] - lower_layer_range[0]
        U = np.sqrt(g_prime * h**3)

        results.append({
            'transect': str(transect),
            "rho1": rho1,
            "rho2": rho2,
            "g'": g_prime,
            "U (m/s)": U,
            "h (m)": h
        })

    return results

results = estimate_critical_flow_speed_all_in_month(
    new_cube,
    target_month="202405",
    along_target=23500,
    upper_layer_range=(0, 100),
    lower_layer_range=(100, 130)
)

for r in results:
    print(r)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

ax.set_xlim(pd.Timestamp("2024-01-01"), pd.Timestamp("2025-06-30"))
ax.set_ylim(24.8, 26.5)

transect_dates = [
    "2024-03-12", "2024-04-16", "2024-05-10", "2024-05-16",
    "2024-07-17", "2024-07-23", "2024-09-18", "2024-09-24",
    "2024-11-14", "2024-11-19", "2025-01-09", "2025-01-14",
    "2025-03-12", "2025-04-30", "2025-05-06", "2025-06-25"]

transect_dates = [
    "2024-03-12", "2024-04-16", "2024-05-10", "2024-05-16",
    "2024-07-17", "2024-07-23", "2024-09-18", "2024-09-24",
    "2024-11-14", "2024-11-19", "2025-01-09", "2025-01-14",
    "2025-03-12", "2025-04-30", "2025-05-06", "2025-06-25"]

rho_divide = np.array([
    np.nan, 25.0, 25.6, 25.5,
    25.6, np.nan, np.nan, np.nan,
    np.nan, np.nan, np.nan, np.nan,
    np.nan, np.nan, np.nan, np.nan])

transect_times = pd.to_datetime(transect_dates)

# Label axes
ax.set_xlabel("Date", fontsize=14)
ax.plot(transect_times, rho_divide, marker='o', linestyle='-', color='blue', label=r'$\rho_{\mathrm{divide}}$ (sill)')
ax.set_ylabel("Density (kg/m³)", fontsize=14)
ax.set_title("Density Time Series", fontsize=16)

ax.tick_params(labelsize=12)
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
import xarray as xr
import numpy as np
from datetime import datetime

# Load your dataset
new_cube = xr.open_dataset('~/Desktop/Summer 2025 Python/calvert_cube.nc')

# ─── Step 1: Find a transect from April 2024 ───────────────────────────
april_transects = [t for t in new_cube.transect.values if str(t).startswith("202404")]
if not april_transects:
    raise ValueError("No April 2024 transects found.")

transect = april_transects[0]
print(f"Using transect: {transect}")

# ─── Step 2: Subset to along = 0–23500 m ───────────────────────────────
ds = new_cube.sel(transect=transect).sel(along=slice(0, 23500))
density = ds['potential_density'].values  # assume already sigma0
depth = ds['depth'].values
along = ds['along'].values

# ─── Step 3: Build 2D coordinate grids ─────────────────────────────────
depths_2d = np.repeat(depth[:, None], len(along), axis=1)
alongs_2d = np.repeat(along[None, :], len(depth), axis=0)

# ─── Step 4: Flatten and filter to valid points ────────────────────────
flat_density = density.ravel()
flat_depths = depths_2d.ravel()
flat_alongs = alongs_2d.ravel()

valid_mask = ~np.isnan(flat_density)
flat_density = flat_density[valid_mask]
flat_depths = flat_depths[valid_mask]
flat_alongs = flat_alongs[valid_mask]

# ─── Step 5: Sort by depth and pick deepest 10 ─────────────────────────
sorted_indices = np.argsort(flat_depths)[::-1]
top_indices = sorted_indices[:10]

# ─── Step 6: Print results ─────────────────────────────────────────────
print("\nTop 10 deepest valid density values (kg/m³):")
for i, idx in enumerate(top_indices):
    val = flat_density[idx]
    dep = flat_depths[idx]
    along_pos = flat_alongs[idx]

    i_depth = np.argmin(np.abs(depth - dep))
    j_along = np.argmin(np.abs(along - along_pos))

    # Compute average over 10 m above
    above_mask = (depth < dep) & (depth >= dep - 10)
    avg_above = np.nan
    if np.any(above_mask):
        above_vals = density[above_mask, j_along]
        if np.any(~np.isnan(above_vals)):
            avg_above = np.nanmean(above_vals)

    print(f"{i+1:2d}.  σθ = {val:.4f} kg/m³   at depth = {dep:.1f} m, along = {along_pos:.1f} m, avg_σθ_above_10m = {avg_above:.4f}")

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ─── Transect dates and rho_divide line ───────────────────────────────
transect_dates = [
    "2024-03-12", "2024-04-16", "2024-05-10", "2024-05-16",
    "2024-07-17", "2024-07-23", "2024-09-18", "2024-09-24",
    "2024-11-14", "2024-11-19", "2025-01-09", "2025-01-14",
    "2025-03-12", "2025-04-30", "2025-05-06", "2025-06-25"]
transect_times = pd.to_datetime(transect_dates)

rho_divide = np.array([
    np.nan, 25.0, 25.6, 25.5,
    25.6, np.nan, np.nan, np.nan,
    np.nan, np.nan, np.nan, np.nan,
    np.nan, np.nan, np.nan, np.nan])

# ─── Initialize storage ───────────────────────────────────────────────
sill_range = (-3000, 7000)
basin_range = (-25500, -10000)
deepest_sill_rho = []
deepest_basin_rho = []
overflow_mean_rho = []

# ─── Loop through transects ───────────────────────────────────────────
for date, rho_thresh in zip(transect_dates, rho_divide):
    date_str = date.replace("-", "")
    matching = [t for t in new_cube.transect.values if str(t).startswith(date_str)]
    
    if not matching:
        deepest_sill_rho.append(np.nan)
        deepest_basin_rho.append(np.nan)
        overflow_mean_rho.append(np.nan)
        continue

    transect = matching[0]
    ds = new_cube.sel(transect=transect)

    def get_deepest_sigma0(region_range):
        region = ds.sel(along=slice(*region_range))
        rho = region['potential_density'].values - 1000
        depth = region['depth'].values
        along = region['along'].values
        depths_2d = np.repeat(depth[:, None], len(along), axis=1)

        valid_mask = ~np.isnan(rho)
        if not np.any(valid_mask):
            return np.nan
        flat_rho = rho[valid_mask]
        flat_depths = depths_2d[valid_mask]
        return flat_rho[np.argmax(flat_depths)]

    deepest_sill_rho.append(get_deepest_sigma0(sill_range))
    deepest_basin_rho.append(get_deepest_sigma0(basin_range))

    if np.isnan(rho_thresh):
        overflow_mean_rho.append(np.nan)
        continue

    sill = ds.sel(along=slice(*sill_range))
    rho = sill['potential_density'].values - 1000
    overflow_mask = rho >= rho_thresh
    overflow_mean_rho.append(np.nanmean(rho[overflow_mask]) if np.any(overflow_mask) else np.nan)

# ─── Plotting ─────────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_xlim(pd.Timestamp("2024-01-01"), pd.Timestamp("2025-06-30"))
ax.set_ylim(24.8, 27)

# Main lines
ax.plot(transect_times, rho_divide, marker='o', linestyle='-', color='blue', label=r'$\rho_{\mathrm{divide}}$')
ax.plot(transect_times, deepest_sill_rho, marker='o', linestyle='-', color='red', label='Sill Bottom Water (-3 to 7 km)')
ax.plot(transect_times, deepest_basin_rho, marker='o', linestyle='-', color='cyan', label='FHS Bottom Water (-25.5 to -10 km)')
ax.plot(transect_times, overflow_mean_rho, marker='o', linestyle='-', color='magenta', label='Overflow Layer Mean')

# Labels
ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel(r"$\sigma_\theta$ (kg m$^{-3}$)", fontsize=14)
ax.set_title("Density Time Series", fontsize=16)
ax.tick_params(labelsize=12)
ax.grid(True)
ax.legend()

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load dataset
new_cube = xr.open_dataset('~/Desktop/Summer 2025 Python/2024_2025_transect_cube.nc')

# ─── Transect dates and rho_divide line ───────────────────────────────
transect_dates = [
    "2024-03-12", "2024-04-16", "2024-05-10", "2024-05-16",
    "2024-07-17", "2024-07-23", "2024-09-18", "2024-09-24",
    "2024-11-14", "2024-11-19", "2025-01-09", "2025-01-14",
    "2025-03-12", "2025-04-30", "2025-05-06", "2025-06-25"]
transect_times = pd.to_datetime(transect_dates)

rho_divide = np.array([
    np.nan, 25.0, 25.6, 25.5,
    25.6, np.nan, np.nan, np.nan,
    np.nan, np.nan, np.nan, np.nan,
    np.nan, np.nan, np.nan, np.nan])

# ─── Initialize storage ───────────────────────────────────────────────
sill_range = (23000, 23500)
basin_range = (0, 15000)
sill_oxy = []
basin_oxy = []
overflow_oxy = []

# ─── Loop through transects ───────────────────────────────────────────
for date, rho_thresh in zip(transect_dates, rho_divide):
    matching = [t for t in new_cube.transect.values if str(t).startswith(date.replace("-", ""))]
    if not matching:
        sill_oxy.append(np.nan)
        basin_oxy.append(np.nan)
        overflow_oxy.append(np.nan)
        continue

    transect = matching[0]
    ds = new_cube.sel(transect=transect)

    def get_deepest_oxygen(region_range):
        region = ds.sel(along=slice(*region_range))
        oxy = region['oxygen_concentration'].values
        depth = region['depth'].values
        along = region['along'].values
        depths_2d = np.repeat(depth[:, None], len(along), axis=1)

        valid_mask = ~np.isnan(oxy)
        if not np.any(valid_mask):
            return np.nan
        flat_oxy = oxy[valid_mask]
        flat_depths = depths_2d[valid_mask]
        return flat_oxy[np.argmax(flat_depths)]

    sill_oxy.append(get_deepest_oxygen(sill_range))
    basin_oxy.append(get_deepest_oxygen(basin_range))

    # ─── Overflow mean oxygen ─────────────────────────────────────────
    if np.isnan(rho_thresh):
        overflow_oxy.append(np.nan)
        continue

    sill = ds.sel(along=slice(*sill_range))
    rho = sill['potential_density'].values - 1000
    oxy = sill['oxygen_concentration'].values

    overflow_mask = rho >= rho_thresh
    if not np.any(overflow_mask):
        overflow_oxy.append(np.nan)
    else:
        overflow_oxy.append(np.nanmean(oxy[overflow_mask]))

# ─── Plotting ─────────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_xlim(pd.Timestamp("2024-01-01"), pd.Timestamp("2025-06-30"))
# ax.set_ylim(0, 7)  # Adjust if your oxygen range is different

ax.scatter(transect_times, sill_oxy, marker='o', linestyle='-', color='red', label='Sill Bottom O₂ (20–30 km)')
ax.scatter(transect_times, basin_oxy, marker='o', linestyle='-', color='cyan', label='FHS Bottom O₂ (0–15 km)')
ax.scatter(transect_times, overflow_oxy, marker='o', linestyle='--', color='magenta', label='Overflow Layer Mean O₂')

ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel("Dissolved O₂ (mL/L)", fontsize=14)
ax.set_title("Oxygen Time Series", fontsize=16)
ax.tick_params(labelsize=12)
ax.grid(True)
ax.legend()
plt.show()